# Egocentric vs. allocentric pose

For the task of pose estimation often the allocentric, instead of the egocentric pose is used.
This is, because when using the egocentric pose different visual features need to be mapped onto the same rotation (a).
Using the allocentric pose alleviates this problem, as visual features do not vary that much (b).

![](https://miro.medium.com/max/720/1*9ngEGXpc_nemUxBYZSE8Ww.png)

Image from [Deep Fitting Degree Scoring Network for Monocular 3D Object Detection](https://arxiv.org/pdf/1904.12681.pdf), found online at https://miro.medium.com/max/720/1*9ngEGXpc_nemUxBYZSE8Ww.png.

<br>

References:
- [3D-RCNN: Instance-level 3D Object Reconstruction via Render-and-Compare](https://openaccess.thecvf.com/content_cvpr_2018/papers/Kundu_3D-RCNN_Instance-Level_3D_CVPR_2018_paper.pdf) 
- [Orientation Estimation in Monocular 3D Object Detection](https://towardsdatascience.com/orientation-estimation-in-monocular-3d-object-detection-f850ace91411)

## Preparation

In the following we define some utility functions. You can skip this part if you just want to fiddle with the visualization.

In [ ]:
# When running in Colab
# %pip install transforms3d trimesh git+https://github.com/skoch9/meshplot.git pythreejs==2.0.0 matplotlib

In [ ]:
from typing import List

import matplotlib.pyplot as plt
import meshplot as mp
import numpy as np
import trimesh
from IPython.display import display, HTML, display_html
from scipy.spatial.transform import Rotation as R
from transforms3d.affines import compose


In [ ]:
mp.website()

def visualize3d(points: List[dict] = None, meshes: List[dict] = None, lines: List[dict] = None):
    """
    Visualize a set of points, lines and meshes. The dictionaries will be passed onto meshplot and need to contain the
    correct plot configuration. 

    Adjusted for use in Colab (see https://github.com/skoch9/meshplot/issues/29)
    """
    ploot = None
    points = points if points is not None else []
    meshes = meshes if meshes is not None else []
    lines = lines if lines is not None else []
    mp.offline = True
    for p in points:
        if ploot is None:
            ploot = mp.plot(**p)
        else:
            ploot.add_points(**p)
    for m in meshes:
        if 'mesh' in m.keys():
            m['v'] = m['mesh'].vertices
            m['f'] = m['mesh'].faces
            del m['mesh']
        elif 'voxel' in m.keys():
            m['v'] = m['voxel'].as_boxes().vertices
            m['f'] = m['voxel'].as_boxes().faces
            del m['voxel']
        if ploot is None:
            ploot = mp.plot(**m)
        else:
            ploot.add_mesh(**m)
    for l in lines:
        if ploot is None:
            ploot = mp.plot(**l)
        else:
            _ = ploot.add_lines(**l)
    return ploot.to_html(imports=True, html_frame=False)

In [ ]:
cmap = plt.get_cmap('jet') 
cube_colors_random = True
if cube_colors_random:
    random_colors = np.array([cmap(i)[:3] for i in np.linspace(0, 1, 12)])  # each triangle a different color
else:
    random_colors = np.array([cmap(0.5)[:3] for i in range(12)])  # uniform color


## Egocentric

The egocentric pose is the orientation of the object w.r.t. the camera coordinate system. 
In the visualization below, our object passes the camera (from right to left) moving in a straight line.
Its rotation w.r.t. to the camera does not change, however, as mentioned before, the appearance changes: On the right we have more information from the front-facing side and on the left more from the back-facing one.

In [ ]:
# Settings
egocentric_boxes = []
extent = 0.25
use_random_rotation = False
if use_random_rotation:
    rotation = R.random().as_matrix()   # define a random rotation
else:
    rotation = np.eye(3)

# Create scene objects
for i in [-2, -1, 0, 1, 2]:
    translation = np.array([i, 0, -1])  # create different translations in `X` direction
    transform = compose(translation, rotation, np.ones((3,)))

    box = trimesh.creation.box(
        extents=[extent]*3,
        transform=transform
    )  # create box with given extent and transformation
    egocentric_boxes.append({"v": np.asarray(box.vertices), "f": np.asarray(box.faces), "c": random_colors})

# Visualize
html = visualize3d(
    meshes=egocentric_boxes, 
    points=[{'v': np.array([[0, 0, 0]]), "shading": {"point_size": 0.4}}]  # camera at position (0, 0, 0)
)
display(HTML(html))
print()

## Allocentric

The allocentric pose refers to the orientation w.r.t. other objects and is sometimes also called orientation or observation angle.
We visualize objects that all have the same allocentric rotation, i.e. the same rotation relative to a ray going through the object's center.

See [3D-RCNN: Instance-level 3D Object Reconstruction via Render-and-Compare](https://openaccess.thecvf.com/content_cvpr_2018/papers/Kundu_3D-RCNN_Instance-Level_3D_CVPR_2018_paper.pdf) Section 4.2, with following notation changes
- `R_egocentric` ~ `R`: Egocentric rotation
- `R_allocentric` ~ `R_v`: Allocentric rotation
- `R_pa_align` ~ `R_c`: Alignment of principal axis and the ray from the camera center through the object center

Other implementations [[1]](https://github.com/siyeonkim33/fetch_manipulation/blob/324d014430af8807e47ca5057743ef2fbc1160b4/src/pose_cnn/lib/utils/se3.py#L32), [[2]](https://github.com/tiantianxuabc/dd3d-supplement/blob/0ad20f5475c45ca5f9009418cef224c009012276/tridet/utils/geometry.py), [[3]](https://github.com/KhiemPhi/Pose-Trans/blob/8ddfa8ce648802699644fe8f4f68062b893e9475/core/utils/utils.py#L97)

In [ ]:
# Settings
if use_random_rotation:
    R_allocentric = R.random()  
else:
    R_allocentric = R.from_matrix(np.eye(3))  # easier to see with eye
# Compute object center positions in a circle
length = 2
eps = 0.1
angle = np.pi * np.linspace(-1 + eps, 0 - eps, 5)
x = np.cos(angle) * length
y = np.zeros_like(x)
z = np.sin(angle) * length
object_centers = np.stack((x, y, z)).T
allocentric_boxes = []


# Create scene objects
for i in range(5):
    principal_axis = np.array([0, 0, 1]).reshape(-1, 3)
    look_at = object_centers[i, ...]
    R_pa_align, _ = R.align_vectors(look_at.reshape(-1, 3), principal_axis)
    R_egocentric = R_pa_align * R_allocentric
    transform = compose(look_at, R_egocentric.as_matrix(), np.ones((3,)))

    box = trimesh.creation.box(
        extents=[extent]*3,
        transform=transform
    )
    box_plot_data = {"v": np.asarray(box.vertices), "f": np.asarray(box.faces), "c": random_colors}
    allocentric_boxes.append(box_plot_data)

# Visualize
html = visualize3d(
    meshes=allocentric_boxes, 
    points=[{'v': np.array([[0, 0, 0]]), "shading": {"point_size": 0.4}}]
)
display(HTML(html))
print()

Note, that the conitnuous 6D pose estimation from [On the Continuity of Rotation Representations in Neural Networks](https://arxiv.org/abs/1812.07035) might be interesting as well. It is implemented in [PyTorch3D](https://pytorch3d.org/) as [matrix_to_rotation_6d](https://pytorch3d.readthedocs.io/en/latest/modules/transforms.html#pytorch3d.transforms.matrix_to_rotation_6d) and [rotation_6d_to_matrix](https://pytorch3d.readthedocs.io/en/latest/modules/transforms.html#pytorch3d.transforms.rotation_6d_to_matrix)